In [ ]:
!git clone https://github.com/Adnlatif/SFI
%cd SFI
!pip install -r RegnskapSFI_requirements.txt --quiet

### STEP 0: Import Libraries

In [4]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import VBox, HBox, Label
from traitlets import HasTraits, Unicode, Instance, observe
import IPython
from tabulate import tabulate
from IPython.display import clear_output

### STEP 1: Importing and Parsing Data
This section import data  (3 Files )for needed for the financial reporting

1. Accounting data from UiO system-Kuben (for now, it uses file is exported but in future it will be directly coupled with Kuben System): (make sure the file has the following foramt)
- Format: .xlxs
- Sheet 1 named to “driftkostnader”
- Sheet 2 named to “lønn”
- Sheet 3 named to “investeringer”

2. SFI internal data: This include
- Format: .xlxs
- Sheet 1 named to “projects”: this contains project codes and names
- Sheet 2 named to “partners”: this contains information about : partner, Inkind hours, hourly rate, total inkind, hours paid to partners by center, hourly rate paid to partners by center, total paid to partners by center, scientific equipment, other operating expense and cash funding by partners

3. Information from NFR: This include. 
- Format: .xlxs
- Sheet 1 named to “nfr_cat”: Operating expense categories defined by NFR
- Sheet 2 named to “nfr_project_cat”: Project categories defined by NFR


Rest of the code, pre-process this data to make usable for further processing.
#### KEEP THE FORMAT STRICTLY AS ABOVE
#### KEEP THE SEQUENCE OF FILE IMPORT AS ABOVE
#### REMEMBER TO FILTER ANY TRANECTION THAT DOESNOT BELONG TO THE CENTRE (SEE THE Prosjekt (T) COLUMN OF THE KUBEN DATA)

#### Reisekostnader is included in the "Driftkost". You can filter "Artsrapport styringskart" by "kurs, konferanser og reiser", or in the "text" box where they will start with "reise". However, the best report for "reisekostnader" is PCP0 where it you can see more details about the travels like when they have travelled, where and why. # That is not implmented yet

In [7]:
file_path_kuben = 'sample_data/demo_kuben_data.xlsx' # Open a file dialog to select the excel file
xl_kuben = pd.read_excel(file_path_kuben, header= 0, sheet_name=None)# Load the excel file
df_dict_kuben = {}# Convert each sheet in the excel file to a dataframe and store them in a dictionary
for sheet_name, df_kuben in xl_kuben.items():
    df_dict_kuben[sheet_name] = df_kuben 

In [8]:
df_kuben

,BA,Bilagsnr,#,Bilagsdato,Periode,Hjelp1,Konto (T),Koststed,Protype (T),Prosjekt,...,Gammelt prosjektnummer,Inntektskategori,Feilføring,Hjlp3,Finansieringskilde,Spes.finkilde,Aktiv,Aktivitetstype,Avsetninger,År
0,M3,90002635,49876,2022-01-31,202201,6050,"Avskrivning på driftsløsøre, inventar, verktøy...",15154500,Bidrag,100097,...,144299.0,Bidrag,0,10038290,NFR,IA,IA,ANNET,IA,2022
1,M3,90002635,44796,2022-01-31,202201,6050,"Avskrivning på driftsløsøre, inventar, verktøy...",15154500,Intern,100000,...,NaN,Bevilgning,0,10028291,KD,RD,ANNEN DRIFT,DRIFT,IA,2022
2,M3,90002635,44795,2022-01-31,202201,6050,"Avskrivning på driftsløsøre, inventar, verktøy...",15154500,Intern,100000,...,NaN,Bevilgning,0,10028144,KD,RD,ANNEN DRIFT,DRIFT,IA,2022
3,M3,90002635,44794,2022-01-31,202201,6050,"Avskrivning på driftsløsøre, inventar, verktøy...",15154500,Intern,100000,...,NaN,Bevilgning,0,10022730,KD,RD,ANNEN DRIFT,DRIFT,IA,2022


In [9]:
#Input file from Kuben system that included Drift kost as well as Salaries
file_path_kuben = 'sample_data/demo_kuben_data.xlsx' # Open a file dialog to select the excel file
xl_kuben = pd.read_excel(file_path_kuben, header= 0, sheet_name=None)# Load the excel file
df_dict_kuben = {}# Convert each sheet in the excel file to a dataframe and store them in a dictionary
for sheet_name, df_kuben in xl_kuben.items():
    df_dict_kuben[sheet_name] = df_kuben 

#Input file from SFI Internal, that included Project Names, contributions from and to Partners
file_path_sfi = 'sample_data/demo_sfi_data.xlsx'
xl_sfi = pd.read_excel(file_path_sfi, header= 0, sheet_name=None)
df_dict_sfi = {}
for sheet_name, df_sfi in xl_sfi.items():
    df_dict_sfi[sheet_name] = df_sfi

#Input file from NFR, that included NFR categories for cost distribution
file_path_nfr = 'sample_data/demo_nfr_data.xlsx'
xl_nfr = pd.read_excel(file_path_nfr, header= 0, sheet_name=None)
df_dict_nfr = {}
for sheet_name, df_nfr in xl_nfr.items():
    df_dict_nfr[sheet_name] = df_nfr

# Combing the two different types of orunning costs (driftkostnader, investeringer) to one cost: "kostnader"
kostnader = pd.concat([df_dict_kuben['driftkostnader'],df_dict_kuben['investeringer']])
kostnader= kostnader.reset_index(drop=True)
kostnader = kostnader[kostnader['Beløp'] >= 0] #Removing all negative Beløp values

lønnskostnader = df_dict_kuben['lønn']
projects_info= df_dict_sfi['projects']
projects = projects_info.sort_values(by='Project_code').iloc[:, :2]
projects_ext_fn = projects_info.sort_values(by='Project_code').iloc[:, 2:]

nfr_runcost_cat=df_dict_nfr['nfr_runcost_cat']
nfr_project_type_cat=df_dict_nfr['nfr_project_cat']
parterns_cont= df_dict_sfi['partners'] #Partners contributaions
partner_type=parterns_cont.iloc[:, [0, -1]]

# Add a row with "None" as the NFR category to the 'nfr_cat' dataframe
nfr_cat_none = {'NFR_cat_code': 0, 'NFR_catagory': 'None'}
nfr_cat = pd.concat([pd.DataFrame(nfr_cat_none, index=[0]), nfr_runcost_cat]).reset_index(drop=True)

### STEP 2: Assigning Properties to the project
This section takes the project names, and the user can interactively assign each of the projects to a specific project category (as defined by the NFR category data input in the previous step)

#### Step 1 : Assigning if the project if it is a collaboration/non collaboration and Innovation/Fundamental
#### Step 2 : Assigning the incentive effect of each project
(*** Incentive effect, 1=Present, 0=Not present. First digit: New R&D activity triggered, Second digit: Increase in size of related R&D activity, Third digit: Enhanced scope of related R&D activity, Fourth digit: Increased speed in execution of related R&D activity)

In [3]:
from IPython.display import display
from ipywidgets import widgets
import pandas as pd

projects_assigned = projects.copy()  # Main dataframe (projects)
options_df_cat_p = nfr_project_type_cat  # NFR categroy dataframe to use as options for the drop-down (Project Names)

row_height = '39px'  # Calculate the height of each row in the Kostnader dataframe

# Function to create a drop-down menu for each row
def create_dropdown_cat_p(row):
    options = ['Nothing is selected yet'] + options_df_cat_p['NFR_Project_Catagory'].tolist()
    dd_cat_p = widgets.Dropdown(options=options)
    dd_cat_p.layout.height = row_height
    return dd_cat_p

# Keep a record of the current selection for each dropdown
current_selections = {}

def record_selection(change):
    # Record the index and new value for each dropdown
    i = dropdowns_cat_p[dropdowns_cat_p == change.owner].index[0]
    current_selections[i] = change.new

dropdowns_cat_p = projects_assigned.apply(create_dropdown_cat_p, axis=1)  # Apply the function to each row

# Assign the record_selection function to the 'value' property for each dropdown
for dd in dropdowns_cat_p:
    dd.observe(record_selection, 'value')

def update_projects_assigned(button):
    # For each selection recorded, update the projects_assigned dataframe
    for i, new_value in current_selections.items():
        projects_assigned.loc[i, 'NFR_Project_Catagory'] = new_value
    # Refresh the dataframe output
    df_widget_cat_p.value = projects_assigned.style.set_table_styles(
        [{'selector': 'td, th', 
          'props': [('white-space', 'nowrap'), ('border', '1px solid grey'), ('border-color', 'grey'), ('padding', '5px')]}]).to_html()

# Create the 'Save' button and assign the update_projects_assigned function to its 'on_click' event
save_button = widgets.Button(description="Save/Refresh")
save_button.on_click(update_projects_assigned)

# Continue with your UI code...

border_style = 'solid 1px #ddd'  # Create the border style

# Wrap the dropdown menus in a VBox widget
dropdowns_box_cat_p = widgets.VBox([widgets.VBox([dd_cat_p], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd_cat_p in dropdowns_cat_p],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto'))

df_widget_cat_p = widgets.HTML(value=projects_assigned.to_html(), layout=widgets.Layout(width='auto', height='auto',box_shadow='none'))  # Wrap the HTML table in an HTML widget

df_box_cat_p = widgets.VBox([df_widget_cat_p], layout=widgets.Layout(border=border_style, padding='1px', width='70%', height='auto',box_shadow='none'))  # Wrap the Kostnader dataframe in a VBox widget

hbox_cat_p = widgets.HBox([dropdowns_box_cat_p, df_box_cat_p], layout=widgets.Layout(height='auto'))  # Create the HBox widget

title_label_cat_p = widgets.Label('Projects Assigned to the NFR Catagories')  # Create a label widget with the title text

# Set the style properties of the title label
title_label_cat_p.layout.margin = '40px 0 0px 0'
title_label_cat_p.layout.font_size = '50px'
title_label_cat_p.layout.font_weight = 'bold'

# Create a label widget with the legend text
legend_label_cat_p = widgets.Label('Assigning if the project if it is a collaboration/non collaboration and Innovation/Fundamental')

# Set the style properties of the legend label
legend_label_cat_p.layout.margin = '10px 0'
legend_label_cat_p.layout.font_size = '50px'
legend_label_cat_p.layout.color = '#777'

# Inserting the save button in the layout
main_box_cat_p = widgets.VBox([title_label_cat_p, hbox_cat_p, save_button, legend_label_cat_p])
display(main_box_cat_p)  # Display the main VBox


In [4]:
# Some Formatting 
# add new columns based on "NFR_Project_Catagory"

if 'NFR_Project_Catagory' in projects_assigned.columns:
    projects_assigned['Collaboration project *'] = projects_assigned['NFR_Project_Catagory'].apply(lambda x: 'No' if isinstance(x, str) and 'Non-Collaboration Project' in x else ('Yes' if isinstance(x, str) and 'Collaboration' in x else np.nan))
    projects_assigned['Type of Research**'] = projects_assigned['NFR_Project_Catagory'].apply(lambda x: 'F' if isinstance(x, str) and 'Fundamental' in x else ('I' if isinstance(x, str) and 'Innovation' in x else np.nan))

# remove the "NFR_Project_Catagory" column
    projects_assigned.drop('NFR_Project_Catagory', axis=1, inplace=True)

# move the new columns to their desired positions
    collab_col = projects_assigned.pop('Collaboration project *')
    projects_assigned.insert(2, 'Collaboration project *', collab_col)

    type_col = projects_assigned.pop('Type of Research**')
    projects_assigned.insert(3, 'Type of Research**', type_col)


else:
    print("Error: Please assign NFR Catagories to the projects/or change slections (atleast one).")



# Adding a list of possible Incentive effect as defined by NFR
incentive_cat = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                number = ''.join([str(i), str(j), str(k), str(l)])
                incentive_cat.append(number)

incentive_cat_df = pd.DataFrame(incentive_cat, columns=["Incentive effect***"])

projects_assigned= projects_assigned.copy()  # Main dataframe (projects)
options_df_cat_p = incentive_cat_df  # NFR category dataframe to use as options for the drop-down (Project Names)

row_height= '39px'  # Calculate the height of each row in the Kostnader dataframe

# Function to create a drop-down menu for each row
def create_dropdown_cat_p(row):
    dd_cat_p = widgets.Dropdown(options=options_df_cat_p['Incentive effect***'].tolist())
    dd_cat_p.layout.height = row_height
    return dd_cat_p

# Keep a record of the current selection for each dropdown
current_selections = {}

def record_selection(change):
    # Record the index and new value for each dropdown
    i = dropdowns_cat_p[dropdowns_cat_p == change.owner].index[0]
    current_selections[i] = change.new

dropdowns_cat_p = projects_assigned.apply(create_dropdown_cat_p, axis=1)  # Apply the function to each row

# Assign the record_selection function to the 'value' property for each dropdown
for dd in dropdowns_cat_p:
    dd.observe(record_selection, 'value')

def update_projects_assigned(button):
    # For each selection recorded, update the projects_assigned dataframe
    for i, new_value in current_selections.items():
        projects_assigned.loc[i, 'Incentive effect***'] = new_value
    # Refresh the dataframe output
    df_widget_cat_p.value = projects_assigned.style.set_table_styles(
        [{'selector': 'td, th', 
          'props': [('white-space', 'nowrap'), ('border', '1px solid grey'), ('border-color', 'grey'), ('padding', '5px')]}]).to_html()

# Create the 'Save' button and assign the update_projects_assigned function to its 'on_click' event
save_button = widgets.Button(description="Save/Refresh")
save_button.on_click(update_projects_assigned)

# Continue with your UI code...

border_style = 'solid 1px #ddd'  # Create the border style

# Wrap the dropdown menus in a VBox widget
dropdowns_box_cat_p = widgets.VBox([widgets.VBox([dd_cat_p], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd_cat_p in dropdowns_cat_p],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto'))

df_widget_cat_p = widgets.HTML(value=projects_assigned.to_html(), layout=widgets.Layout(width='auto', height='auto',box_shadow='none'))  # Wrap the HTML table in an HTML widget

df_box_cat_p = widgets.VBox([df_widget_cat_p], layout=widgets.Layout(border=border_style, padding='1px', width='70%', height='auto',box_shadow='none'))  # Wrap the Kostnader dataframe in a VBox widget

hbox_cat_p = widgets.HBox([dropdowns_box_cat_p, df_box_cat_p], layout=widgets.Layout(height='auto'))  # Create the HBox widget

title_label_cat_p = widgets.Label('Projects Assigned to the NFR Categories')  # Create a label widget with the title text

# Set the style properties of the title label
title_label_cat_p.layout.margin = '40px 0 0px 0'
title_label_cat_p.layout.font_size = '50px'
title_label_cat_p.layout.font_weight = 'bold'

# Create a label widget with the legend text
legend_label_cat_p = widgets.Label('Assigning the incentive effect of each project')

# Set the style properties of the legend label
legend_label_cat_p.layout.margin = '10px 0'
legend_label_cat_p.layout.font_size = '50px'
legend_label_cat_p.layout.color = '#777'

# Inserting the save button in the layout
main_box_cat_p = widgets.VBox([title_label_cat_p, hbox_cat_p, save_button, legend_label_cat_p])
display(main_box_cat_p)  # Display the main VBox


### STEP 3: Assigning Running costs to the individual projects and calculating the total (individual) project cost
This section takes the Running Cost (kostander sheet) and the Pojects sheet and creates a drop-down menu on each cost row with values from the NFR category.

- Step 1 Assign each running cost (driftkonstand) to a specific Project
- Step 2: Assign each running cost (driftkonstand) to a specific NFR catagory
- Step 3: Calculating cost per Project and per NFR catagory

In [5]:
#Step 1 Assign each running cost (driftkonstand) to a specific Project

kostnader_assigned = kostnader.copy()# Main dataframe (Kostnader)
options_df = projects # projects dataframe to use as options for the drop-down (Project Names)
row_height = '39px'# Calculate the height of each row in the Kostnader dataframe

# Initialize the dict to store the current selections
current_selections = {}

# Function to create a drop-down menu for each row of Kostnader
def create_dropdown(row):
    options = ['Nothing is selected yet'] + options_df['Project_Name'].tolist()
    dd = widgets.Dropdown(options=options)
    dd.layout.height = row_height
    return dd

def record_selection(change):
    i = dropdowns[dropdowns == change.owner].index[0]
    current_selections[i] = change.new

def update_kostnader_assigned(button):
    for i, new_value in current_selections.items():
        kostnader_assigned.loc[i, 'Project_Name'] = new_value
    # Specify columns without 'Project_Name'
    columns_to_display = [col for col in kostnader_assigned.columns if col != 'Project_Name']
    df_style_update = kostnader_assigned[columns_to_display].style.set_properties(subset=[], **{'height': row_height})
    df_style_update.set_table_styles([{'selector': 'td, th', 'props': [('white-space', 'nowrap'), 
                                                                ('border', '1px solid grey'), 
                                                                ('border-color', 'grey'), 
                                                                ('padding', '5px')]}])
    df_widget.value = df_style_update.to_html()

dropdowns = kostnader_assigned.apply(create_dropdown, axis=1)
dropdowns.map(lambda dd: dd.observe(record_selection, 'value'))

border_style = 'solid 1px #ddd'# Create the border style
kostnader_assigned.loc[:, 'Project_Name'] = None # Set the value of 'Project_Name' using .loc
df_style = kostnader_assigned.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows

# Update the style to set white-space to nowrap
df_style.set_table_styles([{'selector': 'td, th', 'props': [('white-space', 'nowrap'), 
                                                            ('border', '1px solid grey'), 
                                                            ('border-color', 'grey'), 
                                                            ('padding', '5px')]}])

html_table = df_style.to_html()# Render the styled dataframe as an HTML table
df_widget = widgets.HTML(value=html_table, layout=widgets.Layout(width='auto', height='auto'))# Wrap the HTML table in an HTML widget
df_box = widgets.VBox([df_widget], layout=widgets.Layout(border=border_style, padding='1px', width='67%', height='auto'))# Wrap the Kostnader dataframe in a VBox widget
df_box_height = df_box.layout.height# Get the total height of the df_box widget

# Wrap the dropdown menus in a VBox widget
dropdowns_box = widgets.VBox([widgets.VBox([dd], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd in dropdowns],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto', max_height=df_box_height))


hbox = widgets.HBox([dropdowns_box, df_box], layout=widgets.Layout(height='auto'))# Create the HBox widget
title_label = widgets.Label('Running cost (driftkost) Assigned to Projects')# Create a label widget with the title text

# Set the style properties of the title label
title_label.layout.margin = '40px 0 0px 0'
title_label.layout.font_size = '50px'
title_label.layout.font_weight = 'bold'

legend_label = widgets.Label('Assign each running cost (driftkonstand) to a specific Project')# Create a label widget with the legend text

# Set the style properties of the legend label
legend_label.layout.margin = '10px 0'
legend_label.layout.font_size = '50px'
legend_label.layout.color = '#777'

save_button = widgets.Button(description="Save/Refresh")
save_button.on_click(update_kostnader_assigned)

main_box = widgets.VBox([title_label, hbox, save_button, legend_label])
display(main_box)# Display the main VBox


In [6]:

#Step 2: Assign each running cost (driftkonstand) to a specific NFR catagory

kostnader_assigned_cat= kostnader_assigned.copy()# Main dataframe (Kostnader)
options_df_cat = nfr_runcost_cat# Another dataframe to use as options for the drop-down (NFR Catagories)
row_height= '39px'# Calculate the height of each row in the Kostnader dataframe

# Initialize the dict to store the current selections
current_selections_cat = {}

# Function to create a drop-down menu for each row of Kostnader
def create_dropdown_cat(row):
    options_cat = ['Nothing is selected yet'] + options_df_cat['NFR_catagory'].tolist()
    dd_cat = widgets.Dropdown(options=options_cat)
    dd_cat.layout.height = row_height
    return dd_cat

def record_selection_cat(change):
    i = dropdowns_cat[dropdowns_cat == change.owner].index[0]
    current_selections_cat[i] = change.new

def update_kostnader_assigned_cat(button):
    for i, new_value in current_selections_cat.items():
        kostnader_assigned_cat.loc[i, 'NFR_catagory_1'] = new_value
    columns_to_display = [col for col in kostnader_assigned.columns if col != 'Project_Name']
    df_widget.value = kostnader_assigned[columns_to_display].style.set_properties(subset=[], **{'height': row_height}).to_html()

dropdowns_cat = kostnader_assigned_cat.apply(create_dropdown_cat, axis=1)
dropdowns_cat.map(lambda dd: dd.observe(record_selection_cat, 'value'))

border_style = 'solid 1px #ddd'# Create the border style
kostnader_assigned_cat.loc[:, 'NFR_catagory'] = None # Set the value of 'NFR_catagory' using .loc
df_style_cat = kostnader_assigned_cat.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows

# Update the style to set white-space to nowrap
df_style_cat.set_table_styles([{'selector': 'td, th', 'props': [('white-space', 'nowrap'), 
                                                            ('border', '1px solid grey'), 
                                                            ('border-color', 'grey'), 
                                                            ('padding', '5px')]}])

html_table_cat = df_style_cat.to_html()# Render the styled dataframe as an HTML table
df_widget_cat = widgets.HTML(value=html_table_cat, layout=widgets.Layout(width='auto', height='auto',box_shadow='none'))# Wrap the HTML table in an HTML widget
df_box_cat = widgets.VBox([df_widget_cat], layout=widgets.Layout(border=border_style, padding='1px', width='70%', height='auto',box_shadow='none'))# Wrap the Kostnader dataframe in a VBox widget
df_box_cat_height = df_box_cat.layout.height# Get the total height of the df_box widget

# Wrap the dropdown menus in a VBox widget
dropdowns_box_cat = widgets.VBox([widgets.VBox([dd_cat], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd_cat in dropdowns_cat],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto', max_height=df_box_cat_height))

hbox_cat = widgets.HBox([dropdowns_box_cat, df_box_cat], layout=widgets.Layout(height='auto'))# Create the HBox widget

title_label_cat = widgets.Label('Driftkost Assigned to NFR Catagory')# Create a label widget with the title text

# Set the style properties of the title label
title_label_cat.layout.margin = '40px 0 0px 0'
title_label_cat.layout.font_size = '50px'
title_label_cat.layout.font_weight = 'bold'

# Create a label widget with the legend text
legend_label_cat = widgets.Label('Assign each running cost (driftkonstand) to a specific NFR catagory')

# Set the style properties of the legend label
legend_label_cat.layout.margin = '10px 0'
legend_label_cat.layout.font_size = '50px'
legend_label_cat.layout.color = '#777'

save_button_cat = widgets.Button(description="Save/Refresh")
save_button_cat.on_click(update_kostnader_assigned_cat)

main_box_cat = widgets.VBox([title_label_cat, hbox_cat, save_button_cat, legend_label_cat])
display(main_box_cat)# Display the main VBox


In [7]:
#Step 3: Calculating cost per Project and per NFR catagory

# Calculating cost per Project
cost_vs_project=kostnader_assigned_cat[["Beløp","Project_Name"]]
cost_vs_project = cost_vs_project.groupby(["Project_Name"]).Beløp.sum().reset_index()

# Calculating cost per NFR Catagrogy
cost_vs_nfr_cat=kostnader_assigned_cat[["Beløp","NFR_catagory_1"]]
cost_vs_nfr_cat = cost_vs_nfr_cat.groupby(["NFR_catagory_1"]).Beløp.sum().reset_index()
cost_vs_nfr_cat = cost_vs_nfr_cat.rename(columns={'NFR_catagory_1': 'NFR_catagory'})
SFI_Running_Cost_Cat=cost_vs_nfr_cat
display(cost_vs_project, cost_vs_nfr_cat)


,Project_Name,Beløp
0,Centre Administration,49.00
1,Centre Cultivation,368.82
2,"Equipment, Infrastructure and Laboratory",5003.15
3,Geological Assistant Project Development,1765.27
4,Not Allocated to any project,1610.59
5,OBDA- SIRIUS Optique Pilot,836.67
6,Semantic Plugin for OSDU,406.97
7,Stakeholder engagement,819.09


,NFR_catagory,Beløp
0,Andre personalkostnader,819.09
1,Diverse kostnader,368.82
2,"Konsulenttjenester, web årsrapporter etc",1610.59
3,Kontigenter,836.67
4,Kontorhold,406.97
5,Kurs og Seminarer,49.00
6,Lisenser og programvare,1765.27
7,Reisekostnader,5003.15


### STEP 4: Assigning Personal costs to the individual projects and calculating the total (individual) project cost
This section takes the Personal Cost (from the Kuben data) and the Pojects sheet and creates a drop-down menu on each cost row with values from the Projects.

- Step 1: Defining the max number of person-hours/year
- Step 2: Enter the updatd Overhead/year for a 100% possition e.g. from TDI model:
- Step 3: Get the perosnal (Salaries) data from the Kuben data
- Step 3: Setting up Contract percentages of each employee (names extracted from Kuben file where cost is assigned as Lønn)
- Step 4: Assigning percentage from the employee's total contract percentage to the SFI core budget
- Step 5 : Assigning employee time to the individual projects: take the project names and the total number of hours available for each employee, and the user can use dropdown menus for each project & person for hours distribution.

In [8]:
#Step 1: Defining the max number of person-hours/year

#drop down menu for slecting the total number of hours in a year
min_hours=1400
max_hours=2000

work_hours = list(range(min_hours, max_hours+1))
dropdown_wh = widgets.Dropdown(options=work_hours)

title_wh = widgets.Label(value="Select the number of person-hours/year:")# create a label widget with the title
container_wh = widgets.HBox(children=[title_wh, dropdown_wh])# arrange the label and the drop-down menu horizontally

display(container_wh)# display the container widget

# define a function to update the selected value
def update_selected_wh(change):
    global selected_wh
    selected_wh = change.new

selected_wh = dropdown_wh.value# initialize the selected value
dropdown_wh.observe(update_selected_wh, names='value')# observe the drop-down menu for changes


#Step 2: Enter the updatd Overhead/year for a 100% possition e.g. from TDI model:

min_oh = 0# set the minimum hours
input_oh = widgets.IntText(min=min_oh, max=None, value=0)# create the text field widget with no maximum value
title_oh = widgets.Label(value="Enter the updatd Overhead/year for a 100% possition e.g. from TDI model:")# create a label widget with the title
container_oh = widgets.HBox(children=[title_oh, input_oh])# arrange the label and the text field horizontally
display(container_oh)# display the container widget

# define a function to update the selected value
def update_selected_oh(change):
    global selected_oh
    selected_oh = change.new

selected_oh = input_oh.value# initialize the selected value to 0
input_oh.observe(update_selected_oh, names='value')# observe the text field for changes


In [9]:
#Step3: Get the perosnal (Salaries) data from the Kuben data (df_dict_kuben['lønn'])

#Get the Data
salaries = lønnskostnader.groupby(['Anlegg/ ansattnr (T)']).Beløp.sum().reset_index()
salaries = salaries.sort_values('Anlegg/ ansattnr (T)')# Sort people by name

#Creating Dataframe for percentages
values_per = list(range(0, 101))# Create a list of values from 1 to 100
df_per = pd.DataFrame({'Values': values_per})# Create a dataframe with one column and 100 rows

# Create a function to add the percentage sign to each value
def add_percent_sign(value):
    return '{}%'.format(value)

df_per['Percentage'] = df_per['Values'].apply(add_percent_sign)# Apply the function to each value in the 'Values' column to create the 'Value and %' column
df_per['Values']=df_per['Values']/100# Divinding actual values by 100 to get the muiltiplying factor


#Step 4: Setting up Contract percentages of each employee (names extracted from Kuben file where cost is assigned as Lønn)

personal_cost_c= salaries.copy()# Main dataframe (Salaries)
options_df_per_c = df_per# Another dataframe to use as options for the drop-down (Percentages)
row_height= '39px'# Calculate the height of each row in the dataframe

# Function to create a drop-down menu for each row
def create_dropdown_per_c(row):
    dd_per_c = widgets.Dropdown(options=options_df_per_c['Percentage'].tolist())
    dd_per_c.observe(update_personal_cost_c, 'value')
    dd_per_c.layout.height = row_height
    return dd_per_c

def update_personal_cost_c(change):
    i = dropdowns_per_c[dropdowns_per_c == change.owner].index[0]
    personal_cost_c.loc[i, 'Total Contract %'] = change.new

dropdowns_per_c = personal_cost_c.apply(create_dropdown_per_c, axis=1)# Apply the function to each row
border_style = 'solid 1px #ddd'# Create the border style
df_style_per_c = personal_cost_c.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows

# Update the style to set white-space to nowrap
df_style_per_c.set_table_styles([{'selector': 'td, th', 'props': [('white-space', 'nowrap'), 
                                                            ('border', '1px solid grey'), 
                                                            ('border-color', 'grey'), 
                                                            ('padding', '5px')]}])

html_table_per_c = df_style_per_c.to_html()# Render the styled dataframe as an HTML table
df_widget_per_c= widgets.HTML(value=html_table_per_c, layout=widgets.Layout(width='auto', height='auto',box_shadow='none'))# Wrap the HTML table in an HTML widget
df_box_per_c = widgets.VBox([df_widget_per_c], layout=widgets.Layout(border=border_style, padding='1px', width='70%', height='auto',box_shadow='none'))# Wrap the Kostnader dataframe in a VBox widget
df_box_per_height_c = df_box_per_c.layout.height# Get the total height of the df_box widget

# Wrap the dropdown menus in a VBox widget
dropdowns_box_per_c = widgets.VBox([widgets.VBox([dd_per], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd_per in dropdowns_per_c],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto', max_height=df_box_per_height_c))

hbox_per_c = widgets.HBox([dropdowns_box_per_c, df_box_per_c], layout=widgets.Layout(height='auto'))# Create the HBox widget
title_label_per_c = widgets.Label('Employee Total Contract with the Institute: %')# Create a label widget with the title text

# Set the style properties of the title label
title_label_per_c.layout.margin = '40px 0 0px 0'
title_label_per_c.layout.font_size = '50px'
title_label_per_c.layout.font_weight = 'bold'

# Create a label widget with the legend text
legend_label_per_c = widgets.Label('This is the Total Pecentage for a person in the contract.')

# Set the style properties of the legend label
legend_label_per_c.layout.margin = '10px 0'
legend_label_per_c.layout.font_size = '50px'
legend_label_per_c.layout.color = '#777'

main_box_per_c = widgets.VBox([title_label_per_c, hbox_per_c,legend_label_per_c])
display(main_box_per_c)# Display the main VBox


In [10]:
# Error Check

if 'Total Contract %' not in personal_cost_c.columns:
    print("Error: 'Please Assign the percentage for employees (at least one person) first")
else:
    personal_cost_c

In [11]:
#Step 4: Assigning percentage from the employee's total contract percentage to the SFI core budget

personal_cost= personal_cost_c.copy()# Main dataframe (Salaries)
personal_cost= personal_cost[['Anlegg/ ansattnr (T)','Beløp','Total Contract %']]
options_df_per = df_per# Another dataframe to use as options for the drop-down (Percentages)
row_height= '39px'# Calculate the height of each row in the dataframe

# Function to create a drop-down menu for each row
def create_dropdown_per(row):
    dd_per = widgets.Dropdown(options=options_df_per['Percentage'].tolist())
    dd_per.observe(update_personal_cost, 'value')
    dd_per.layout.height = row_height
    return dd_per

def update_personal_cost(change):
    i = dropdowns_per[dropdowns_per == change.owner].index[0]
    personal_cost.loc[i, 'Percentage Charged to the core budjet'] = change.new

dropdowns_per = personal_cost.apply(create_dropdown_per, axis=1)# Apply the function to each row
border_style = 'solid 1px #ddd'# Create the border style
df_style_per = personal_cost.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows

# Update the style to set white-space to nowrap
df_style_per.set_table_styles([{'selector': 'td, th', 'props': [('white-space', 'nowrap'), 
                                                            ('border', '1px solid grey'), 
                                                            ('border-color', 'grey'), 
                                                            ('padding', '5px')]}])
html_table_per = df_style_per.to_html()# Render the styled dataframe as an HTML table
df_widget_per= widgets.HTML(value=html_table_per, layout=widgets.Layout(width='auto', height='auto',box_shadow='none')) # Wrap the HTML table in an HTML widget
df_box_per = widgets.VBox([df_widget_per], layout=widgets.Layout(border=border_style, padding='1px', width='70%', height='auto',box_shadow='none'))# Wrap the Kostnader dataframe in a VBox widget
df_box_per_height = df_box_per.layout.height # Get the total height of the df_box widget

# Wrap the dropdown menus in a VBox widget
dropdowns_box_per = widgets.VBox([widgets.VBox([dd_per], layout=widgets.Layout(border=border_style, padding='auto', height=row_height)) for dd_per in dropdowns_per],
                             layout=widgets.Layout(width='33%', height='auto', padding='1px', margin='41.8px 0 0 0', overflow_y='auto', max_height=df_box_per_height))

# Create the HBox widget
hbox_per = widgets.HBox([dropdowns_box_per, df_box_per], layout=widgets.Layout(height='auto'))
title_label_per = widgets.Label('Employee Percentage charged to the core budget')# Create a label widget with the title text

# Set the style properties of the title label
title_label_per.layout.margin = '40px 0 0px 0'
title_label_per.layout.font_size = '50px'
title_label_per.layout.font_weight = 'bold'

# Create a label widget with the legend text
legend_label_per = widgets.Label('Assigning percentage from the employee total contract percentage to the SFI core budget.')

# Set the style properties of the legend label
legend_label_per.layout.margin = '10px 0'
legend_label_per.layout.font_size = '50px'
legend_label_per.layout.color = '#777'

main_box_per = widgets.VBox([title_label_per,hbox_per,legend_label_per])
display(main_box_per)# Display the main VBox

In [12]:
# Error Check and Processing of data

if 'Percentage Charged to the core budjet' not in personal_cost.columns:
    print("Error: 'Please Assign the percentage for persons (atleast one person) first")
else:
    # iterate over each row of the dataframe
    personal_cost['Percentage Charged to the core budjet'] = personal_cost['Percentage Charged to the core budjet'].fillna('0%')

    for index, row in personal_cost.iterrows():
        if float(str(row['Percentage Charged to the core budjet']).rstrip("%")) > float(str(row['Total Contract %']).rstrip("%")):
            print(f"Percentage charged to the budget is larger than the contract for {row['Anlegg/ ansattnr (T)']}.")
        else:
            personal_cost['Percentage_X'] = personal_cost['Percentage Charged to the core budjet'].str.rstrip('%').fillna(0).astype('int')/100
            personal_cost['Percentage_XX'] = personal_cost_c['Total Contract %'].str.rstrip('%').fillna(0).astype('int')/100
            personal_cost['Beløp_with_overhead']=personal_cost['Beløp']+(personal_cost['Percentage_XX']*selected_oh)
            personal_cost['PerHourCost']= personal_cost['Beløp_with_overhead']/(personal_cost['Percentage_XX']*selected_wh)
            personal_cost['Percentage_Adjusted_salary'] = personal_cost['Beløp_with_overhead']*personal_cost['Percentage_X']/personal_cost['Total Contract %'].str.rstrip('%').fillna(0).astype('int')*100
            personal_cost_percentage_adjusted = personal_cost[['Anlegg/ ansattnr (T)','Beløp','Total Contract %','Percentage Charged to the core budjet','Percentage_Adjusted_salary']]
            personal_cost_temp=personal_cost[['Anlegg/ ansattnr (T)','Beløp','Beløp_with_overhead','Total Contract %','Percentage Charged to the core budjet','Percentage_Adjusted_salary','PerHourCost']]
    print("Things look good for this section! The personal cost is now processed further.")

Things look good for this section! The personal cost is now processed further.


In [237]:
# Processing of the data with the information provided above
# Calcualting Percentage Adjusted salaries as well as the number of hours changed to the core budget 

personal_cost_percentage_adjusted_copy = personal_cost_percentage_adjusted.copy()# create a copy of the DataFrame
personal_cost_percentage_adjusted_copy['Percentage_Adjusted_salary'] = personal_cost_percentage_adjusted_copy['Percentage_Adjusted_salary'].astype(int)

personal_cost_percentage_adjusted_copy.loc[:, 'Number of Hours'] = personal_cost['Percentage_X'].apply(lambda x: x * selected_wh)# use the .loc indexer to add a new column 'Number of Hours' with values based on a calculation
personal_cost_percentage_adjusted_copy['Number of Hours'] = personal_cost_percentage_adjusted_copy['Number of Hours'].astype(int)
personal_cost_percentage_adjusted = personal_cost_percentage_adjusted_copy


personal_hours=personal_cost_percentage_adjusted[['Anlegg/ ansattnr (T)','Number of Hours']]

personal_hours_cal=personal_hours.copy()
personal_hours_cal = personal_hours_cal.rename(columns={'Anlegg/ ansattnr (T)': 'Person', 'Number of Hours': 'Total Number of Hours'})
personal_hours_cal= personal_hours_cal.set_index('Person')
personal_hours_cal=personal_hours_cal.transpose()

# another dataframe called "other_hours" with a "Total Number of Hours" column
other_hours_series = projects["Project_Name"]# extract the "Total Number of Hours" column as a Pandas Series object
personal_hours_cal = pd.concat([personal_hours_cal, other_hours_series], axis=1)# concatenate the other_hours_series to "personal_hours_cal"
cols = personal_hours_cal.columns.tolist()# get the column names in a list
cols = [cols[-1]] + cols[:-1]# move the last column to the first position in the list

personal_hours_cal = personal_hours_cal[cols]# re-order the dataframe columns using the updated list of column names
personal_hours_cal = personal_hours_cal.rename(columns={'Project_Name': ''})
personal_hours_cal = personal_hours_cal.reset_index(drop=True)
personal_hours_cal.loc[0, ''] = 'Project Name / Number of Hours available'

personal_hours_cal_ = personal_hours_cal.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows


# Step 5 : Assigning employee time to the individual projects: take the project names and the total number of 
# hours available for each employee, and the user can use dropdown menus for each project & person for hours 
# distribution.

df_personal_hours_cal =personal_hours_cal# Define the DataFrame
df_personal_hours_cal = df_personal_hours_cal.fillna(-1)# Replacing NaN value tih paceholder -1 (for performance)

# Define the custom function to create HTML select elements for NaN values
def dropdown_for_nan(val):
    if val == -1:
        return f'<select onchange="store_selection(this)">' + ''.join(f'<option value="{i}">{i}</option>' for i in range(selected_wh+1)) + '</select>'
    else:
        return val

# Define the JavaScript function to store selections in an object
JS = """
<script>
  var selections_pers = {};

  // Check if selections_pers exists before creating a new one
  if (typeof selections_pers !== 'undefined') {
    delete selections_pers;
  }

  function store_selection(select) {
    var row = select.parentNode.parentNode;
    var col = select.parentNode.cellIndex;
    var val = select.options[select.selectedIndex].value;
    var name = row.cells[0].innerHTML;
    if (!selections_pers[name]) {
        selections_pers[name] = {};
    }
    selections_pers[name][col] = val;
  }

  function save_selections() {
    var kernel = IPython.notebook.kernel;

    // Delete existing selections_pers before saving
    if (typeof selections_pers !== 'undefined') {
      delete selections_pers;
    }

    kernel.execute('selections_pers = ' + JSON.stringify(selections_pers));
  }
</script>
"""


df_personal_hours_cal = df_personal_hours_cal.where(pd.notnull(df_personal_hours_cal), None)# Replace NaN values in the DataFrame with None so that they can be styled as strings
df_personal_hours_cal_dropdowns = df_personal_hours_cal.applymap(dropdown_for_nan)# Apply the custom function to the DataFrame to create dropdown menus
df_personal_hours_cal_html = df_personal_hours_cal_dropdowns.to_html(escape=False)# Generate the HTML table without using Styler
html_personal_hours_cal = JS + df_personal_hours_cal_html + '<button onclick="save_selections()">Save</button>'# Combine the JavaScript and HTML code

display(HTML(html_personal_hours_cal))# Display the HTML code


###### If you save once but make changes and forget to save it again, this will take the old selections in calcuation: 

In [267]:
# ERROR CHECK

try:
    selections_pers
except NameError:
    print("You forget to save your selections.")
else:
    print("I have the saved data, but please click Save one more time. After hitting a Save, ignore this message ")


I have the saved data, but please click Save one more time. After hitting a Save, ignore this message 


In [268]:
# Further Calculations and Varifictiaion if all the available hours (for SFI core budget)are allocated to projects

df_personal_hours_cal_2=df_personal_hours_cal

# define a function to replace -1 with 0, except in column 1 ## replacning all -1 vlaues to zero
def replace_minus_one(col):
    if col.name != '':
        return col.apply(lambda x: 0 if x == -1 else x)
    else:
        return col
df_personal_hours_cal_2 = df_personal_hours_cal_2.apply(replace_minus_one)# apply the function to each column in the DataFrame

selections_unnested = {(int(row), int(col)-1): float(val) for row, col_val in selections_pers.items() for col, val in col_val.items()}# un-nested selections
for (row, col), value in selections_unnested.items():# loop over the dictionary and update the DataFrame
    df_personal_hours_cal_2.iloc[row, col] = value

df_personal_hours_cal_2 = df_personal_hours_cal_2.fillna(0)# replace NaN values with 0

## This block sum all the assigned hours by drop-down,and add it to the main data frame
sum_df_personal_hours_cal_2 = df_personal_hours_cal_2.iloc[1:, :].sum()# Compute the sum of df1, excluding the first row

# create a new dataframe with the sum_df1 values and the same column names as df1
df_personal_hours_cal_3 = pd.DataFrame(sum_df_personal_hours_cal_2.values.reshape(1, -1), columns=df_personal_hours_cal_2.columns)
df_personal_hours_cal_4 = pd.concat([df_personal_hours_cal_2.iloc[:1], df_personal_hours_cal_3, df_personal_hours_cal_2.iloc[1:]]).reset_index(drop=True)
df_personal_hours_cal_4.iloc[1, 0] = 'Hours Allocated'
df_personal_hours_cal_4.iloc[0, 0] = 'Total Number of hours for Allocation'

# df_personal_hours_cal_4  is a data frame with hours, allocated by dropdowns, and  a new row added at row index 1with the sum of all entries in a column
# This block checks if the total number of hours assigned through drop-down are equal (Green), less(Blud) or more (Red)

df_personal_hours_cal_5=df_personal_hours_cal_4.iloc[0:2, :]

diff_row_df_personal_hours_cal_5 = df_personal_hours_cal_5.iloc[:,1:].loc[0, :] -df_personal_hours_cal_5.iloc[:,1:].loc[1, :]# select the two rows to subtract and compute the difference
diff_df_personal_hours_cal_5 = pd.DataFrame([diff_row_df_personal_hours_cal_5], columns=df_personal_hours_cal_5.columns)# create a new DataFrame with the difference as a row

df_concat_df_personal_hours_cal_5 = pd.concat([df_personal_hours_cal_5, diff_df_personal_hours_cal_5], ignore_index=True)# concatenate the original DataFrame and the new DataFrame
df_concat_df_personal_hours_cal_5.iloc[2, 0] = 'Remaining Hours'


df_personal_hours_cal_6=df_concat_df_personal_hours_cal_5.iloc[2:, :]# create a sample DataFrame
df_personal_hours_cal_6=df_personal_hours_cal_6.reset_index(drop=True)

 #function to apply color and text style to cells
def color_negative_red(val):
    bg_color = '#f86368' if val < 0 else '#3582c4' if val > 0 else '#00ba37'
    text_color = 'white'
    border_style = 'solid'
    border_width = '1px'
    return f'background-color: {bg_color}; color: {text_color}; text-align: center; border-style: {border_style}; border-width: {border_width};'
df_personal_hours_cal_6_styled_df = df_personal_hours_cal_6.style.applymap(color_negative_red, df_personal_hours_cal_6.columns[1:])# create a style object and apply the color function to the selected rows

print ("Red: Number of Hours assigned are greater than the available hours for the person")
print ("Blue: Number of Hours assigned are less than the available hours for the person")
print ("Green: Number of Hours assigned are equal to the available hours for the person")

df_personal_hours_cal_6_styled_df # display the styled dataframe


Red: Number of Hours assigned are greater than the available hours for the person
Blue: Number of Hours assigned are less than the available hours for the person
Green: Number of Hours assigned are equal to the available hours for the person


,,"Aplha,Charli","Beta,Bravo","Gamma,drove"
0,Remaining Hours,0.000000,0.000000,0.000000


In [269]:
#Total Personal Cost/Proejct

## calculating per hour cost of each personal cost person
personal_perhour=personal_cost_temp[['Anlegg/ ansattnr (T)','PerHourCost']]
personal_perhour=personal_perhour.transpose()
personal_perhour.columns = personal_perhour.iloc[0]
personal_perhour= personal_perhour.drop(personal_perhour.index[0])


host_paid_per_project=df_personal_hours_cal_2.replace(-1, 0)
# iterate over all columns of host_paid_per_project (except the first column)
for column in host_paid_per_project.columns[1:]:
    # extract the corresponding value from the first row of personal_perhour
    value = personal_perhour.iloc[0][column]
    # multiply all values in the current column of df1 with the extracted value
    host_paid_per_project[column] = host_paid_per_project[column].apply(lambda x: x * value)

#host_paid_per_project=host_paid_per_project.iloc[1:, :].reset_index(drop=True)
host_paid_per_project['Host Institute Personal Cost'] = host_paid_per_project.iloc[:, 1:].sum(axis=1)
host_paid_per_project = host_paid_per_project[['', 'Host Institute Personal Cost']]
host_paid_per_project.loc[host_paid_per_project.index[0], host_paid_per_project.columns[0]] = 'Total Cost'
host_paid_per_project['Host Institute Personal Cost'] = host_paid_per_project['Host Institute Personal Cost'].apply(lambda x: '{:.0f}'.format(x))

#host_paid_per_project #Total Personal Cost/Proejct

### STEP 5 : Partners Contributions - In-kind
- Step 1: Getting In-Kind data from the SFI Data File, alredy imported in the start
- Step 2 : Assigning partners Inkind time contributaions to the individual projects: take the project names and the total number of hours available for each partner, and the user can use dropdown menus for each project & partner for hours distribution.


In [22]:
#Step 1: Get data from the and process it for further analysis # "parterns_cont_inkind" dataframe, and pre-processing for further analysis

parterns_cont_inkind=parterns_cont.iloc[:, 0:2].transpose()
parterns_cont_inkind.columns = parterns_cont_inkind.iloc[0]
parterns_cont_inkind = parterns_cont_inkind.drop(parterns_cont_inkind.index[0])

row_height="39px"
other_hours_series = projects["Project_Name"]# extract the "Total Number of Hours" column as a Pandas Series object
parterns_cont_inkind = pd.concat([parterns_cont_inkind, other_hours_series], axis=1)# concatenate the other_hours_series to "personal_hours_cal"
cols = parterns_cont_inkind.columns.tolist()# get the column names in a list
cols = [cols[-1]] + cols[:-1]# move the last column to the first position in the list

parterns_cont_inkind = parterns_cont_inkind[cols]# re-order the dataframe columns using the updated list of column names
parterns_cont_inkind = parterns_cont_inkind.rename(columns={'Project_Name': ''})
parterns_cont_inkind = parterns_cont_inkind.reset_index(drop=True)
parterns_cont_inkind.loc[0, ''] = 'Project Name / Number of Hours available'

parterns_cont_inkind_ = parterns_cont_inkind.style.set_properties(subset=[], **{'height': row_height})# Set the cell height of the rows


In [175]:
# drop down menu for slecting the total number of hours in a year : used only for creating the dropdown limit

min_hours_p=0
max_hours_p=5000

hours_p = list(range(min_hours_p, max_hours_p+1))
dropdown_p = widgets.Dropdown(options=hours_p)

# create a label widget with the title
title_p = widgets.Label(value="Select the max number of hours/per project (used only for creating the dropdown limit, chosing a very high number will affect the performance):")
container_p = widgets.HBox(children=[title_p, dropdown_p])# arrange the label and the drop-down menu horizontally
display(container_p)# display the container widget

# define a function to update the selected value
def update_selected_p(change):
    global selected_p
    selected_p = change.new

selected_p = dropdown_p.value# initialize the selected value
dropdown_p.observe(update_selected_p, names='value')# observe the drop-down menu for changes


In [179]:
## This script takes the dataframe with proejct names, total number of hourse available for resrchers,
## and create dropdown menues for each proejct & individual person.
# Define the DataFrame
df_parterns_cont_inkind =parterns_cont_inkind
df_parterns_cont_inkind = df_parterns_cont_inkind.fillna(-1)# Replacing NaN value tih paceholder -1 (for performance)

# Define the custom function to create HTML select elements for NaN values
def dropdown_for_nan(val):
    if val == -1:
        return f'<select onchange="store_selection(this)">' + ''.join(f'<option value="{i}">{i}</option>' for i in range(selected_p)) + '</select>'
    else:
        return val

# Define the JavaScript function to store selections in an object
JS = """
<script>
  var selections_part = {};

  // Check if selections_part exists before creating a new one
  if (typeof selections_part !== 'undefined') {
    delete selections_part;
  }

  function store_selection(select) {
    var row = select.parentNode.parentNode;
    var col = select.parentNode.cellIndex;
    var val = select.options[select.selectedIndex].value;
    var name = row.cells[0].innerHTML;
    if (!selections_part[name]) {
        selections_part[name] = {};
    }
    selections_part[name][col] = val;
  }

  function save_selections() {
    var kernel = IPython.notebook.kernel;

    // Delete existing selections_part before saving
    if (typeof selections_part !== 'undefined') {
      delete selections_part;
    }

    kernel.execute('selections_part = ' + JSON.stringify(selections_part));
  }
</script>
"""


# Replace NaN values in the DataFrame with None so that they can be styled as strings
df_parterns_cont_inkind = df_parterns_cont_inkind.where(pd.notnull(df_parterns_cont_inkind), None)

# Apply the custom function to the DataFrame to create dropdown menus
df_parterns_cont_inkind_dropdowns = df_parterns_cont_inkind.applymap(dropdown_for_nan)

# Generate the HTML table without using Styler
df_parterns_cont_inkind_html = df_parterns_cont_inkind_dropdowns.to_html(escape=False)

# Combine the JavaScript and HTML code
html_parterns_cont_inkind = JS + df_parterns_cont_inkind_html + '<button onclick="save_selections()">Save / Refreash</button>'

# Display the HTML code
display(HTML(html_parterns_cont_inkind))



In [222]:
import IPython

try:
    # Replace "my_variable" with the name of the variable you want to check for
    selections_part
except NameError:
    print("You forget to save your selections")
else:
    # This code will run if the variable was found
    print("I have the saved data, but it might be the old one. Please click Save/Refresh one more time ")
    


I have the saved data, but it might be the old one. Please click Save/Refresh one more time 


In [223]:
## this block is for Gettting slections out as dict,uncpaking it and replace it in the df_personal_hours_cal. 
#the new dataframe is called df_personal_hours_cal_2

df_parterns_cont_inkind_2=df_parterns_cont_inkind.copy()

## replacning all -1 vlaues to zero

# define a function to replace -1 with 0, except in column 1
def replace_minus_one(col):
    if col.name != '':
        return col.apply(lambda x: 0 if x == -1 else x)
    else:
        return col

# apply the function to each column in the DataFrame
df_parterns_cont_inkind_2 = df_parterns_cont_inkind_2.apply(replace_minus_one)

# un-nested selections
selections_unnested = {(int(row), int(col)-1): float(val) for row, col_val in selections_part.items() for col, val in col_val.items()}
# loop over the dictionary and update the DataFrame
for (row, col), value in selections_unnested.items():
    df_parterns_cont_inkind_2.iloc[row, col] = value


## This block sum all the assgined hours by drop down,and add it to the main data frame
# compute the sum of df1 excluding the first row
sum_df_parterns_cont_inkind_2 = df_parterns_cont_inkind_2.iloc[1:, :].sum()

# create a new dataframe with the sum_df1 values and the same column names as df1
df_parterns_cont_inkind_3 = pd.DataFrame(sum_df_parterns_cont_inkind_2.values.reshape(1, -1), columns=df_parterns_cont_inkind_2.columns)
df_parterns_cont_inkind_4 = pd.concat([df_parterns_cont_inkind_2.iloc[:1], df_parterns_cont_inkind_3, df_parterns_cont_inkind_2.iloc[1:]]).reset_index(drop=True)
df_parterns_cont_inkind_4.iloc[1, 0] = 'Hours Allocated'
df_parterns_cont_inkind_4.iloc[0, 0] = 'Total Number of hours for Allocation'
#
## df_parterns_cont_inkind_4  is dataframe with hourse, allocated by dropdowns, and  a new row added at row index 1
## with sum of all entries in a column

### This block check if the total number of hours assigned through drop down are 
## equal (Green), less(Blud) or more (Red)

df_parterns_cont_inkind_5=df_parterns_cont_inkind_4.iloc[0:2, :]

# select the two rows to subtract and compute the difference
diff_row_df_parterns_cont_inkind_5 = df_parterns_cont_inkind_5.iloc[:,1:].loc[0, :] -df_parterns_cont_inkind_5.iloc[:,1:].loc[1, :]

# create a new DataFrame with the difference as a row
diff_df_parterns_cont_inkind_5 = pd.DataFrame([diff_row_df_parterns_cont_inkind_5], columns=df_parterns_cont_inkind_5.columns)

# concatenate the original DataFrame and the new DataFrame
df_concat_df_parterns_cont_inkind_5 = pd.concat([df_parterns_cont_inkind_5, diff_df_parterns_cont_inkind_5], ignore_index=True)

df_concat_df_parterns_cont_inkind_5.iloc[2, 0] = 'Remaining Hours'

# create a sample DataFrame
df_parterns_cont_inkind_6=df_concat_df_parterns_cont_inkind_5.iloc[2:, :]
df_parterns_cont_inkind_6=df_parterns_cont_inkind_6.reset_index(drop=True)

# define a function to apply color and text style to cells
def color_negative_red(val):
    bg_color = '#f86368' if val < 0 else '#3582c4' if val > 0 else '#00ba37'
    text_color = 'white'
    border_style = 'solid'
    border_width = '1px'
    return f'background-color: {bg_color}; color: {text_color}; text-align: center; border-style: {border_style}; border-width: {border_width};'

# create a style object and apply the color function to the selected rows
df_parterns_cont_inkind_6_styled_df = df_parterns_cont_inkind_6.style.applymap(color_negative_red, df_parterns_cont_inkind_6.columns[1:])

# display the styled dataframe
df_parterns_cont_inkind_6_styled_df

,,Partner 1,Partner 2,Partner 3,Partner 4,Partner 5,Partner 6
0,Remaining Hours,0,0,0,0,0,0


### STEP 6 : Partners Contributions - Paid To Partners
- Step 1: Getting Paid To Partners data from the SFI Data File, alredy imported in the start
- Step 2 : Assigning partners Paid To Partners time  to the individual projects: take the project names and the total number of hours available for each partner (Paid To Partners), and the user can use dropdown menus for each project & partner for hours distribution.

In [32]:
# get data from the "parterns_cont_inkind" dataframe, and pre processing for further analysis
parterns_cont_paid=parterns_cont.iloc[:, [0,4]].transpose()
parterns_cont_paid.columns = parterns_cont_paid.iloc[0]
parterns_cont_paid = parterns_cont_paid.drop(parterns_cont_paid.index[0])
parterns_cont_paid = parterns_cont_paid.loc[:, (parterns_cont_paid != 0).any(axis=0)] # Removning parterns that do not have not been paid

row_height="39px"

# assume you have another dataframe called "other_hours" with a "Total Number of Hours" column
# extract the "Total Number of Hours" column as a Pandas Series object
other_hours_series = projects["Project_Name"]

# concatenate the other_hours_series to "personal_hours_cal"
parterns_cont_paid = pd.concat([parterns_cont_paid, other_hours_series], axis=1)

# assume you have a dataframe called "my_dataframe"
# get the column names in a list
cols = parterns_cont_paid.columns.tolist()

# move the last column to the first position in the list
cols = [cols[-1]] + cols[:-1]

# re-order the dataframe columns using the updated list of column names
parterns_cont_paid = parterns_cont_paid[cols]

parterns_cont_paid = parterns_cont_paid.rename(columns={'Project_Name': ''})
parterns_cont_paid = parterns_cont_paid.reset_index(drop=True)
parterns_cont_paid.loc[0, ''] = 'Project Name / Number of Hours available'

# Set the cell height of the rows
parterns_cont_paid_ = parterns_cont_paid.style.set_properties(subset=[], **{'height': row_height})


In [33]:
#drop donw manue for slecting total number of hours in an year
min_hours_pp=0
max_hours_pp=10000

hours_pp = list(range(min_hours_pp, max_hours_pp+1))
dropdown_pp = widgets.Dropdown(options=hours_pp)

# create a label widget with the title
title_pp = widgets.Label(value="Select the max number of hours/per proejct (chose a very high number will effect the performance):")

# arrange the label and the drop-down menu horizontally
container_pp = widgets.HBox(children=[title_pp, dropdown_pp])

# display the container widget
display(container_pp)

# define a function to update the selected value
def update_selected_pp(change):
    global selected_pp
    selected_pp = change.new

# initialize the selected value
selected_pp = dropdown_pp.value

# observe the drop-down menu for changes
dropdown_pp.observe(update_selected_pp, names='value')

In [309]:
## This script takes the dataframe with proejct names, total number of hourse available for resrchers,
## and create dropdown menues for each proejct & individual person.
# Define the DataFrame
df_parterns_cont_paid =parterns_cont_paid
df_parterns_cont_paid = df_parterns_cont_paid.fillna(-1)# Replacing NaN value tih paceholder -1 (for performance)

# Define the custom function to create HTML select elements for NaN values
def dropdown_for_nan(val):
    if val == -1:
        return f'<select onchange="store_selection(this)">' + ''.join(f'<option value="{i}">{i}</option>' for i in range(selected_pp)) + '</select>'
    else:
        return val

# Define the JavaScript function to store selections in an object
JS = """
<script>
  var selections_part_paid = {};

  // Check if selections_part_paid exists before creating a new one
  if (typeof selections_part_paid !== 'undefined') {
    delete selections_part_paid;
  }

  function store_selection(select) {
    var row = select.parentNode.parentNode;
    var col = select.parentNode.cellIndex;
    var val = select.options[select.selectedIndex].value;
    var name = row.cells[0].innerHTML;
    if (!selections_part_paid[name]) {
        selections_part_paid[name] = {};
    }
    selections_part_paid[name][col] = val;
  }

  function save_selections() {
    var kernel = IPython.notebook.kernel;

    // Delete existing selections_part_paid before saving
    if (typeof selections_part_paid !== 'undefined') {
      delete selections_part_paid;
    }

    kernel.execute('selections_part_paid = ' + JSON.stringify(selections_part_paid));
  }
</script>
"""



# Replace NaN values in the DataFrame with None so that they can be styled as strings
df_parterns_cont_paid = df_parterns_cont_paid.where(pd.notnull(df_parterns_cont_paid), None)


# Apply the custom function to the DataFrame to create dropdown menus
df_parterns_cont_paid_dropdowns = df_parterns_cont_paid.applymap(dropdown_for_nan)

# Generate the HTML table without using Styler
df_parterns_cont_paid_html = df_parterns_cont_paid_dropdowns.to_html(escape=False)

# Combine the JavaScript and HTML code
html_parterns_cont_paid = JS + df_parterns_cont_paid_html + '<button onclick="save_selections()">Save / Refresh</button>'

# Display the HTML code
display(HTML(html_parterns_cont_paid))



In [312]:
try:
   
    selections_part_paid
except NameError:
    print("You forget to save your selections")
else:
    # This code will run if the variable was found
    print("I have the saved data, but it might be the old one. Please click Save/Refresh one more time ")

I have the saved data, but it might be the old one. Please click Save/Refresh one more time 


In [313]:
## this block is for Gettting slections out as dict,uncpaking it and replace it in the df_personal_hours_cal. 
#the new dataframe is called df_personal_hours_cal_2

df_parterns_cont_paid_2=df_parterns_cont_paid

## replacning all -1 vlaues to zero

# define a function to replace -1 with 0, except in column 1
def replace_minus_one(col):
    if col.name != '':
        return col.apply(lambda x: 0 if x == -1 else x)
    else:
        return col

# apply the function to each column in the DataFrame
df_parterns_cont_paid_2 = df_parterns_cont_paid_2.apply(replace_minus_one)

# un-nested selections
selections_unnested = {(int(row), int(col)-1): float(val) for row, col_val in selections_part_paid.items() for col, val in col_val.items()}

# loop over the dictionary and update the DataFrame
for (row, col), value in selections_unnested.items():
    df_parterns_cont_paid_2.iloc[row, col] = value

# replace NaN values with 0
df_parterns_cont_paid_2 = df_parterns_cont_paid_2.fillna(0)

## This block sum all the assgined hours by drop down,and add it to the main data frame
# compute the sum of df1 excluding the first row
sum_df_parterns_cont_paid_2 = df_parterns_cont_paid_2.iloc[1:, :].sum()

# create a new dataframe with the sum_df1 values and the same column names as df1
df_parterns_cont_paid_3 = pd.DataFrame(sum_df_parterns_cont_paid_2.values.reshape(1, -1), columns=df_parterns_cont_paid_2.columns)
df_parterns_cont_paid_4 = pd.concat([df_parterns_cont_paid_2.iloc[:1], df_parterns_cont_paid_3, df_parterns_cont_paid_2.iloc[1:]]).reset_index(drop=True)
df_parterns_cont_paid_4.iloc[1, 0] = 'Hours Allocated'
df_parterns_cont_paid_4.iloc[0, 0] = 'Total Number of hours for Allocation'
#
## df_parterns_cont_paid_4  is dataframe with hourse, allocated by dropdowns, and  a new row added at row index 1
## with sum of all entries in a column

### This block check if the total number of hours assigned through drop down are 
## equal (Green), less(Blud) or more (Red)

df_parterns_cont_paid_5=df_parterns_cont_paid_4.iloc[0:2, :]

# select the two rows to subtract and compute the difference
diff_row_df_parterns_cont_paid_5 = df_parterns_cont_paid_5.iloc[:,1:].loc[0, :] -df_parterns_cont_paid_5.iloc[:,1:].loc[1, :]

# create a new DataFrame with the difference as a row
diff_df_parterns_cont_paid_5 = pd.DataFrame([diff_row_df_parterns_cont_paid_5], columns=df_parterns_cont_paid_5.columns)

# concatenate the original DataFrame and the new DataFrame
df_concat_df_parterns_cont_paid_5 = pd.concat([df_parterns_cont_paid_5, diff_df_parterns_cont_paid_5], ignore_index=True)

df_concat_df_parterns_cont_paid_5.iloc[2, 0] = 'Remaining Hours'

# create a sample DataFrame
df_parterns_cont_paid_6=df_concat_df_parterns_cont_paid_5.iloc[2:, :]
df_parterns_cont_paid_6=df_parterns_cont_paid_6.reset_index(drop=True)

# define a function to apply color and text style to cells
def color_negative_red(val):
    bg_color = '#f86368' if val < 0 else '#3582c4' if val > 0 else '#00ba37'
    text_color = 'white'
    border_style = 'solid'
    border_width = '1px'
    return f'background-color: {bg_color}; color: {text_color}; text-align: center; border-style: {border_style}; border-width: {border_width};'

# create a style object and apply the color function to the selected rows
df_parterns_cont_paid_6_styled_df = df_parterns_cont_paid_6.style.applymap(color_negative_red, df_parterns_cont_paid_6.columns[1:])

# display the styled dataframe
df_parterns_cont_paid_6_styled_df

,,Partner 6
0,Remaining Hours,0


### STEP 7: The Total Cost Per Project ( SFI Annual Accounting Report - Project Characteristics and Costs)

In [314]:
# Further Processing: Just run this cell
partners_paid_NOK=parterns_cont.iloc[:, [0,4,6]].copy()
partners_paid_NOK['Hourly rate for partners- Paid']=partners_paid_NOK['total paid to partners by centre']/partners_paid_NOK['hours paid to partners by centre']
partners_paid_per_hr= partners_paid_NOK[['partner','Hourly rate for partners- Paid']].fillna(0)
partners_paid_per_hr= partners_paid_per_hr.transpose()
partners_paid_per_hr

new_header = partners_paid_per_hr.iloc[0] # get the first row as header
partners_paid_per_hr = partners_paid_per_hr[1:] # remove the first row
partners_paid_per_hr.columns = new_header # set the column index
partners_paid_per_hr=partners_paid_per_hr.reset_index(drop=True)

partners_paid_per_project_hr=df_parterns_cont_paid_2.replace(-1, 0)
partners_paid_per_project_hr=partners_paid_per_project_hr.iloc[1:, :].reset_index(drop=True)

#mulitping with hourly rate to get total cost
partners_paid_per_project_NOK=partners_paid_per_project_hr.copy()
partners_paid_per_hr

# Identify common column names
common_cols = set(partners_paid_per_project_NOK.columns) & set(partners_paid_per_hr.columns)

# Loop over common column names and multiply values
for col in common_cols:
    if col in partners_paid_per_hr.columns:
        multiplier = partners_paid_per_hr.loc[0, col]  # Extract first row value from df2
        partners_paid_per_project_NOK[col] = partners_paid_per_project_NOK[col] * multiplier  # Multiply df1 column with the extracted value


##Calcuating the total Host Cost/Project

# Step 1: Claculating total host PERSONAL COST per project
#calculating per hour cost of each personal cost person
personal_perhour=personal_cost_temp[['Anlegg/ ansattnr (T)','PerHourCost']]
personal_perhour=personal_perhour.transpose()
personal_perhour.columns = personal_perhour.iloc[0]
personal_perhour= personal_perhour.drop(personal_perhour.index[0])


host_paid_per_project=df_personal_hours_cal_2.replace(-1, 0)
# iterate over all columns of host_paid_per_project (except the first column)
for column in host_paid_per_project.columns[1:]:
    # extract the corresponding value from the first row of personal_perhour
    value = personal_perhour.iloc[0][column]
    host_paid_per_project[column] = host_paid_per_project[column].apply(lambda x: x * value)# multiply all values in the current column of df1 with the extracted value

#host_paid_per_project=host_paid_per_project.iloc[1:, :].reset_index(drop=True)
host_paid_per_project['Host Institute Personal Cost'] = host_paid_per_project.iloc[:, 1:].sum(axis=1)
host_paid_per_project = host_paid_per_project[['', 'Host Institute Personal Cost']]
host_paid_per_project.loc[host_paid_per_project.index[0], host_paid_per_project.columns[0]] = 'Total Cost'
host_paid_per_project['Host Institute Personal Cost'] = host_paid_per_project['Host Institute Personal Cost'].apply(lambda x: '{:.0f}'.format(x))


# Step 2: Claculating total host COST PAID TO PARTNERS per project


### Cost per proejct from perosnal cost
paid_to_personal_pp=host_paid_per_project

### Cost per proejct from driftkost
host_drifkost_per_project = pd.merge(projects, cost_vs_project[['Project_Name', 'Beløp']], on='Project_Name',how='left').fillna(0)
host_drifkost_per_project['Host Institute Running Cost']=host_drifkost_per_project['Beløp']
host_drifkost_per_project=host_drifkost_per_project[['Project_code','Project_Name','Host Institute Running Cost']]
paid_to_running_pp=host_drifkost_per_project

## Summng Drift+Personl
drift_personal_cost_per_project = pd.merge(paid_to_running_pp, paid_to_personal_pp, left_on='Project_Name', right_on='')
drift_personal_cost_per_project = drift_personal_cost_per_project.drop('', axis=1) # drop the matching column and rename the added column


######## NOTE:THIS NEED TO BE CALARIDIED WIRH ØYVIND
### For now cost for the host institutie which is paid to the partners is excluded from the clacuation. and that amount will add to the cost of that particular partner
## summing with the paid to partners cost
#drift_personal_partner_paid_cost_per_project =pd.merge(drift_personal_cost_per_project, paid_to_patners_pp, left_on='Project_Name', right_on='')
#drift_personal_partner_paid_cost_per_project =drift_personal_partner_paid_cost_per_project.drop('', axis=1)

## Total Host cost 
total_host_cost= drift_personal_cost_per_project
total_host_cost['Host Institute Running Cost'] = total_host_cost['Host Institute Running Cost'].astype(int)
total_host_cost['Host Institute Personal Cost'] = total_host_cost['Host Institute Personal Cost'].astype(int)
sum_col = total_host_cost.iloc[:, 2:5].sum(axis=1)

# create a new column for the sum and drop columns at index 2, 3, and 4
total_host_cost = total_host_cost.assign(Sum=sum_col).drop(total_host_cost.columns[2:5], axis=1)
total_host_cost = total_host_cost.rename(columns={'Sum': 'Total Cost for the Host Institute'})


##### calcuating In kind Costs/ per project (cost to the partners that is paid by the center is also added to it, see the comments above)

##In-kind in NOK
partners_inkind_NOK=parterns_cont.iloc[:, [0,1,3]].copy()
partners_inkind_NOK['Hourly rate for partners- Inkind']=partners_inkind_NOK['total inkind']/partners_inkind_NOK['inkind hours']
partners_inkind_per_hr= partners_inkind_NOK[['partner','Hourly rate for partners- Inkind']].fillna(0)
partners_inkind_per_hr= partners_inkind_per_hr.transpose()
new_header = partners_inkind_per_hr.iloc[0] # get the first row as header

#In-Kind cost per hour
partners_inkind_per_hr = partners_inkind_per_hr[1:] # remove the first row
partners_inkind_per_hr.columns = new_header # set the column index
partners_inkind_per_hr=partners_inkind_per_hr.reset_index(drop=True)

# In-kind Per project in NOK
partners_inkind_per_project_hr=df_parterns_cont_inkind_2.replace(-1, 0)
partners_inkind_per_project_hr=partners_inkind_per_project_hr.iloc[1:, :]#.reset_index(drop=True)

# mulitping with hourly rate to get total cost
partners_inkind_per_project_NOK=partners_inkind_per_project_hr
# Step 1: Identify common column names between df1 and df2
common_cols = set(partners_inkind_per_project_NOK.columns) & set(partners_inkind_per_hr.columns)

# Loop over common column names and multiply values
for col in common_cols:
    if col in partners_inkind_per_hr.columns:
        multiplier = partners_inkind_per_hr.loc[0, col]  # Extract first row value from df2
        partners_inkind_per_project_NOK[col] = partners_inkind_per_project_NOK[col] * multiplier  # Multiply df1 column with the extracted value
        
total_inkind_cost=partners_inkind_per_project_NOK

#Summing Paid and In-kind cost
partners_paid_per_project_NOK_1= partners_paid_per_project_NOK.copy()
partners_inkind_per_project_NOK_1=partners_inkind_per_project_NOK.copy()

partners_paid_per_project_NOK_1 = partners_paid_per_project_NOK_1.drop(partners_paid_per_project_NOK.columns[0], axis=1)
total_inkind_and_paid_cost=pd.concat([partners_inkind_per_project_NOK.reset_index(drop=True), partners_paid_per_project_NOK_1], axis=1).fillna(0)
total_inkind_and_paid_cost = total_inkind_and_paid_cost.reset_index(drop=True)
total_inkind_and_paid_cost = total_inkind_and_paid_cost.groupby(total_inkind_and_paid_cost.columns, axis=1).sum()
total_inkind_and_paid_cost = total_inkind_and_paid_cost.loc[~(total_inkind_and_paid_cost == 0).all(axis=1)]
total_inkind_and_paid_cost.columns.values[0] = "Project_Name"


#Summing total host cost per project and Inkind cost
total_cost_per_project=pd.merge(total_host_cost, total_inkind_and_paid_cost, left_on='Project_Name', right_on='Project_Name')
total_cost_per_project =total_cost_per_project.drop('Project_Name', axis=1)

# Merging Project properties to the cost 
# Drop the "Project_code" column from df2
projects_assigned_dropped = projects_assigned.drop("Project_code", axis=1)

# Get the columns from df1 before and after the insertion point
total_cost_per_project_columns_before = total_cost_per_project.columns[:2]
total_cost_per_project_columns_after = total_cost_per_project.columns[2:]


# Concatenate df1 columns before, df2_dropped, and df1 columns after
total_cost_per_project = pd.concat([total_cost_per_project[total_cost_per_project_columns_before], projects_assigned_dropped, total_cost_per_project[total_cost_per_project_columns_after]], axis=1)
column_to_move = total_cost_per_project.pop(total_cost_per_project.columns[1])  # Remove the column and retrieve it
total_cost_per_project.insert(5, column_to_move.name, column_to_move)  # Insert the column at the new position

#Formatting a bit to get things in shape
total_cost_per_project= total_cost_per_project.sort_values('Project_code').reset_index(drop=True)

#adding all costs per project to produce the total cost column
# Filter out non-numeric columns
numeric_columns = total_cost_per_project.iloc[:, 2:].select_dtypes(include=[np.number]).columns

# Calculate the total cost per project by summing only numeric columns
total_cost_per_project['Total cost'] = total_cost_per_project[numeric_columns].sum(axis=1)

## moving NOT Allocated to projects to the end of the dataframe
# move the first row to the last
total_cost_per_project = pd.concat([total_cost_per_project.iloc[1:], total_cost_per_project.iloc[0:1]])

# reset the index
total_cost_per_project = total_cost_per_project.reset_index(drop=True)


In [315]:
### SFI Equipment (Software) - Host ----- (Not already included as running cost)

# Claculating cost of Software, paid by the host instituute (Something that is not included in the running cost)

min_sw = 0# set the minimum vlaue that can be added
input_sw = widgets.IntText(min=min_sw, max=None, value=0)# create the text field widget with no maximum value
title_sw = widgets.Label(value="Enter the cost for the host associated with the SFI Equipment (Software)")# create a label widget with the title
container_sw = widgets.HBox(children=[title_sw, input_sw])# arrange the label and the text field horizontally
display(container_sw)# display the container widget

# define a function to update the selected value
def update_selected_sw(change):
    global selected_sw
    selected_sw = change.new

selected_sw = input_sw.value# initialize the selected value to 0
input_sw.observe(update_selected_sw, names='value')# observe the text field for changes

In [316]:
### SFI Equipment (Software) - Partners ----- (Not already reported in terms of in-kind HOURS)
#It will take information from the input data on partners (sfi)

# Claculating cost of Software, paid by the Partners (Something that is not included in the in-kind as hours)

parterns_cont_inkind_software=parterns_cont.iloc[:, [0,7]].transpose()
parterns_cont_inkind_software.columns = parterns_cont_inkind_software.iloc[0]
parterns_cont_inkind_software = parterns_cont_inkind_software.drop(parterns_cont_inkind_software.index[0])

sfi_equipment=parterns_cont_inkind_software
sfi_equipment['Total Cost for the Host Institute'] =selected_sw

sfi_equipment['Project_code']=0
sfi_equipment['Project_Name']='SFI Equipment (Software)'

# Move the last column to index 1
cols = list(sfi_equipment.columns)
cols = [cols[-1]] + cols[:-1]
sfi_equipment = sfi_equipment[cols]

# Move the last column to index 1
cols = list(sfi_equipment.columns)
cols = [cols[-1]] + cols[:-1]
sfi_equipment = sfi_equipment[cols]

# Move the last column to index 2
cols = list(sfi_equipment.columns)
cols = cols[:-1] + [cols[-1]]
cols = cols[:2] + [cols[-1]] + cols[2:-1]
sfi_equipment = sfi_equipment[cols]


sfi_equipment = sfi_equipment.reset_index(drop=True)
sfi_equipment.index.name = None

In [317]:
### SFI Administration -Host ----- (Not already reported as running cost)

# set the minimum hours
min_admin = 0

# create the text field widget with no maximum value
input_admin = widgets.IntText(min=min_admin, max=None, value=0)

# create a label widget with the title
title_admin = widgets.Label(value="Enter the cost for the host SFI Administration (only direct costs - labour in Project 001)")

# arrange the label and the text field horizontally
container_admin = widgets.HBox(children=[title_admin, input_admin])

# display the container widget
display(container_admin)

# define a function to update the selected value
def update_selected_admin(change):
    global selected_admin
    selected_admin = change.new

# initialize the selected value to 0
selected_admin = input_admin.value

# observe the text field for changes
input_admin.observe(update_selected_admin, names='value')

In [318]:
### SFI Administration - Partners ----- (Not already reported in terms of in-kind HOURS)
#It will take information from the input data on partners (sfi)

# Adding SFi Administration from partners to the SFI administration from the host Institute

parterns_cont_inkind_admin=parterns_cont.iloc[:, [0,8]].transpose()
parterns_cont_inkind_admin.columns = parterns_cont_inkind_admin.iloc[0]
parterns_cont_inkind_admin = parterns_cont_inkind_admin.drop(parterns_cont_inkind_admin.index[0])

sfi_admin=parterns_cont_inkind_admin
sfi_admin['Total Cost for the Host Institute'] =selected_admin

sfi_admin['Project_code']=0
sfi_admin['Project_Name']='SFI Administration (only direct costs - labour in Project 001)'

# Move the last column to index 1
cols = list(sfi_admin.columns)
cols = [cols[-1]] + cols[:-1]
sfi_admin = sfi_admin[cols]

# Move the last column to index 1
cols = list(sfi_admin.columns)
cols = [cols[-1]] + cols[:-1]
sfi_admin = sfi_admin[cols]

# Move the last column to index 2
cols = list(sfi_admin.columns)
cols = cols[:-1] + [cols[-1]]
cols = cols[:2] + [cols[-1]] + cols[2:-1]
sfi_admin = sfi_admin[cols]


sfi_admin = sfi_admin.reset_index(drop=True)
sfi_admin.index.name = None

In [319]:

cost_temp = pd.concat([total_cost_per_project, sfi_equipment], axis=0)
cost_temp_2=pd.concat([cost_temp, sfi_admin], axis=0)
cost_temp_2 = cost_temp_2.drop(cost_temp_2.columns[-1], axis=1)


# Divide columns by 1000 and round off to 0 decimal points
cost_temp_2.iloc[:, 5:] = cost_temp_2.iloc[:, 5:].applymap(lambda x: round(x/1000, 0))


#Adding Total Costs row in the end
# Calculate the sum of values in each column except the first two
sum_row_cost_temp_2 = cost_temp_2.iloc[:, 5:].sum()

# Create a new row with the sums
new_row_cost_temp_2 = pd.DataFrame([[0, 'Total costs'] + sum_row_cost_temp_2.values.tolist()], columns=['Project_code', 'Project_Name'] + sum_row_cost_temp_2.index.tolist())

# Concatenate the original DataFrame with the new row
NFR_cost = pd.concat([cost_temp_2, new_row_cost_temp_2], ignore_index=True)
NFR_cost['Total Project Cost'] = NFR_cost.iloc[:, 5:].sum(axis=1)
NFR_cost.iloc[:, 5:]=NFR_cost.iloc[:, 5:].applymap(lambda x: '{:.0f}'.format(x))
for col in NFR_cost.columns[5:]:
    NFR_cost[col] = pd.to_numeric(NFR_cost[col], errors='coerce')
SFI_cost= NFR_cost

In [320]:
print ('SFI Annual Accounting Report - Project Characteristics and Costs (All figures in 1000 NOK)')
SFI_cost

SFI Annual Accounting Report - Project Characteristics and Costs (All figures in 1000 NOK)


,Project_code,Project_Name,Collaboration project *,Type of Research**,Incentive effect***,Total Cost for the Host Institute,Partner 1,Partner 2,Partner 3,Partner 4,Partner 5,Partner 6,Total Project Cost
0,1,Centre Administration,NaN,NaN,0001,246,107,0,0,0,0,0,353
1,2,"Equipment, Infrastructure and Laboratory",NaN,NaN,0010,5,0,0,0,0,0,0,5
2,1002,Geological Assistant Project Development,Yes,I,0010,287,107,60,0,33,110,1,598
3,1101,Semantic Plugin for OSDU,No,I,0100,275,214,0,0,0,0,1178,1667
4,1102,OBDA- SIRIUS Optique Pilot,Yes,F,0100,176,1,0,0,0,0,36,213
5,1103,SIRIUS Subsurface Data Laboratory,No,F,0100,293,54,0,0,0,110,0,457
6,5001,Stakeholder engagement,NaN,NaN,0100,211,54,0,0,33,0,0,298
7,5004,Centre Cultivation,NaN,NaN,0101,145,21,0,0,0,55,0,221
8,0,Not Allocated to any project,NaN,NaN,NaN,2,0,0,0,0,0,0,2
9,0,SFI Equipment (Software),NaN,NaN,NaN,0,0,60,50,0,0,0,110


### STEP 8: Funding Distributaion (based on the budget)

In [102]:
# Funding Distribution

min_fn = 0# set the minimum vlaue that can be added
input_fn = widgets.IntText(min=min_fn, max=None, value=0)# create the text field widget with no maximum value
title_fn = widgets.Label(value="Total Funding Available from NFR")# create a label widget with the title
container_fn = widgets.HBox(children=[title_fn, input_fn])# arrange the label and the text field horizontally
display(container_fn)# display the container widget

# define a function to update the selected value
def update_selected_fn(change):
    global selected_fn
    selected_fn = change.new

selected_fn = input_fn.value# initialize the selected value to 0
input_fn.observe(update_selected_fn, names='value')# observe the text field for changes¨


In [321]:
nfr=selected_fn/1000
paid_column_sums = partners_paid_per_project_NOK_1.astype(int).sum()# Sum all the values along each column
paid = paid_column_sums.sum()/1000# Sum the resulting series to get the total sum

SFI_eq_administration= NFR_cost.iloc[-2:-4:-1, -1].sum()
non_hours_cost_host= NFR_cost.iloc[1, 5].astype(int) + NFR_cost.iloc[-2:-5:-1, 5].astype(int).sum()

inkind = NFR_cost.iloc[-1, 6:-1].astype(int).sum()-paid-SFI_eq_administration
cash = parterns_cont['cash fuding by partners'].sum()/1000
host=(float(NFR_cost.iloc[-1, 5])+ paid-nfr) - cash


# Create a list of lists representing the table data
table_data = [
    ["Total Funding by NFR", f"{nfr}"],
    ["Total Funding by Host Institute (Own Financing)", f"{host}"],
    ["Total Inkind Contribution", f"{inkind}"],
    ["Total SFI Admin and Equipment Contribution", f"{SFI_eq_administration}"],
    ["Total Paid Contribution", f"{paid}"],
    ["Total Cash Contribution", f"{cash}"],
    ["Total Cost", f"{cash + inkind + host + nfr + SFI_eq_administration}"]]

# Display the table using tabulate
table = tabulate(table_data, headers=["Funding", "Amount in KNOK"], tablefmt="fancy_grid",stralign="right")
print(table)

╒═════════════════════════════════════════════════╤══════════════════╕
│                                         Funding │   Amount in KNOK │
╞═════════════════════════════════════════════════╪══════════════════╡
│                            Total Funding by NFR │         1500     │
├─────────────────────────────────────────────────┼──────────────────┤
│ Total Funding by Host Institute (Own Financing) │          533.644 │
├─────────────────────────────────────────────────┼──────────────────┤
│                       Total Inkind Contribution │         1030.36  │
├─────────────────────────────────────────────────┼──────────────────┤
│      Total SFI Admin and Equipment Contribution │          110     │
├─────────────────────────────────────────────────┼──────────────────┤
│                         Total Paid Contribution │         1143.64  │
├─────────────────────────────────────────────────┼──────────────────┤
│                         Total Cash Contribution │          750     │
├─────

### STEP 9: Funding (SFI Annual Accounting Report - Funding)

In [322]:
#Adding Hours from In-kind and Paid
#Adding Hours from In-kind, Paid and then Host Institute

# Get the partner names from df1 that are present in df2
common_columns = [col for col in df_parterns_cont_inkind_2.iloc[1:].columns if col in partners_paid_per_project_hr.columns]

# Create df3 by copying all columns from df1
partners_inkind_paid_per_project_hr = df_parterns_cont_inkind_2.iloc[1:].copy()

# Iterate over common columns and sum the values from df2
for col in common_columns:
    # Check if the column is numeric (not a string)
    if pd.api.types.is_numeric_dtype(df_parterns_cont_inkind_2.iloc[1:][col]):
        partners_inkind_paid_per_project_hr[col] = df_parterns_cont_inkind_2.iloc[1:][col] + partners_paid_per_project_hr[col]


# Get the partner names from df1 that are present in df2
common_columns = [col for col in df_parterns_cont_inkind_2.iloc[1:].columns if col in partners_paid_per_project_hr.columns]

# Create df3 by copying all columns from df1
partners_inkind_paid_per_project_hr = df_parterns_cont_inkind_2.iloc[1:].copy()

 #Iterate over common columns and sum the values from df2
for col in common_columns:
    # Check if the column is numeric (not a string)
    if pd.api.types.is_numeric_dtype(df_parterns_cont_inkind_2.iloc[1:][col]):
        partners_inkind_paid_per_project_hr[col] = df_parterns_cont_inkind_2.iloc[1:][col] + partners_paid_per_project_hr[col]

host_hours=df_personal_hours_cal_2[1:].copy()
# Convert columns to numeric type where possible
host_hours = host_hours.apply(pd.to_numeric, errors='ignore')

# Filter numeric columns
numeric_cols = host_hours.select_dtypes(include=[float, int]).columns

# Add values from numeric columns and create a new column
host_hours['Total Hours for the host institute'] = host_hours[numeric_cols].sum(axis=1)

# Drop columns that were added together
host_hours = host_hours.drop(columns=numeric_cols)

#adding host hours to partners hours
host_partners_inkind_paid_per_project_hr = pd.merge(partners_inkind_paid_per_project_hr, host_hours, on='', how='inner')

# Output the merged dataframe
# Replacing any NAN with zero
host_partners_inkind_paid_per_project_hr = host_partners_inkind_paid_per_project_hr.fillna(0)
# moving "Total Hours for the host institute" to possition two in the dataframe
host_partners_inkind_paid_per_project_hr = host_partners_inkind_paid_per_project_hr[[host_partners_inkind_paid_per_project_hr.columns[0]] + [host_partners_inkind_paid_per_project_hr.columns[-1]] + host_partners_inkind_paid_per_project_hr.columns[1:-1].tolist()]
# Select numeric columns
numeric_cols = host_partners_inkind_paid_per_project_hr.select_dtypes(include=[float, int]).columns

# Calculate total project hours
host_partners_inkind_paid_per_project_hr['Total Project Hours'] = host_partners_inkind_paid_per_project_hr[numeric_cols].sum(axis=1)

#host_partners_inkind_paid_per_project_hr= host_partners_inkind_paid_per_project_hr[1:]
host_partners_inkind_per_project_hr_df = host_partners_inkind_paid_per_project_hr.copy()
df_parterns_cont_inkind_2_df = df_parterns_cont_inkind_2.copy()
df_parterns_cont_inkind_2_df =df_parterns_cont_inkind_2_df.drop(df_parterns_cont_inkind_2_df.index[:1])

df_parterns_cont_inkind_2_df.reset_index(drop=True, inplace=True)
host_partners_inkind_per_project_hr_df.reset_index(drop=True, inplace=True)
# Get the intersection of column names
common_columns = host_partners_inkind_per_project_hr_df.columns.intersection(df_parterns_cont_inkind_2_df.columns)

# Replace values in df1 based on matching column names
host_partners_inkind_per_project_hr_df[common_columns] = df_parterns_cont_inkind_2_df[common_columns]

# this takes informaiton about contributions from the SIRIUS sfi file,and combine it with the hours distribution on projects
# Note that the first row is in NOK whereas rest of the rows are in hours

partners_cash_inkind_eq=parterns_cont[['partner','cash fuding by partners','total inkind','other operating expense']].transpose()
# Assign the first row as column names
partners_cash_inkind_eq.columns = partners_cash_inkind_eq.iloc[0]
# Drop the first row
partners_cash_inkind_eq = partners_cash_inkind_eq.iloc[1:].reset_index(drop=True)
new_row = partners_cash_inkind_eq.sum()
# Add the new row to the dataframe using loc
partners_cash_inkind_eq.loc[len(partners_cash_inkind_eq)] = new_row
# Drop all rows except the last row
partners_cash_inkind_eq = partners_cash_inkind_eq.iloc[[-1]]
host_partners_cash_inkind_eq_combined = pd.concat([host_partners_inkind_per_project_hr_df, partners_cash_inkind_eq], sort=False, ignore_index=True)
# Move the last row to the top
host_partners_cash_inkind_eq_combined = pd.concat([host_partners_cash_inkind_eq_combined.iloc[-1:], host_partners_cash_inkind_eq_combined.iloc[:-1]], ignore_index=True)
host_partners_cash_inkind_eq_combined.iat[0, 0] = 'Total contribution NOK'
host_partners_cash_inkind_eq_combined.iat[0, 1] = NFR_cost.iloc[-1, 5].astype(int)*1000
host_partners_cash_inkind_eq_combined = host_partners_cash_inkind_eq_combined.drop('Total Project Hours', axis=1)
host_partners_cash_inkind_eq_combined['Total'] = host_partners_cash_inkind_eq_combined.drop('', axis=1).sum(axis=1)

#Creating NRF fudning dataframe by using formullas
#Host: Sum of Project Hours for the host x Sum of UiO Cost / 1000 / Sum of all Proejcts hours
#Partner :Project hours partner x ((Total Inkind and cash (not paid or software))/1000)/ Sum of in kind hours for a partner

funding_temp_1= NFR_cost.iloc[:-3, :2]

funding_partners_temp_1=host_partners_cash_inkind_eq_combined.copy()
sum_row = funding_partners_temp_1.iloc[1:,1:].sum()
new_row = pd.DataFrame([sum_row], columns=funding_partners_temp_1.columns[1:])# Create a new row with the sum
funding_partners_temp_2 = pd.concat([funding_partners_temp_1, new_row]).reset_index(drop=True)

# Extract the last row
last_row = funding_partners_temp_2.iloc[[-1]]

# Drop the last row from the original dataframe
funding_partners_temp_2 = funding_partners_temp_2.iloc[:-1]

# Insert the last row at index 1
funding_partners_temp_2 = pd.concat([funding_partners_temp_2.iloc[:1], last_row, funding_partners_temp_2.iloc[1:]]).reset_index(drop=True)

funding_partners_temp_2.iat[1, 0] = 'Total Time Contribution'

######### Calculating the Host contribution for each project #####
#Getting the total number of hours for the host institute 
host_partners_cash_inkind_eq_combined_temp=host_partners_cash_inkind_eq_combined.loc[1:]['Total Hours for the host institute'].reset_index(drop=True)
first_row_ = host_partners_cash_inkind_eq_combined_temp.iloc[[0]]# Extract the first row
host_partners_cash_inkind_eq_combined_temp = host_partners_cash_inkind_eq_combined_temp.drop(host_partners_cash_inkind_eq_combined_temp.index[0])# Remove the first row from the dataframe
host_partners_cash_inkind_eq_combined_temp = pd.concat([host_partners_cash_inkind_eq_combined_temp, first_row_], ignore_index=True)# Append the extracted first row to the dataframe

funding_temp_1['Host Institute']=(host_partners_cash_inkind_eq_combined_temp*host)/funding_partners_temp_2.iloc[1, 1]

funding_partners_temp_2_slice = funding_partners_temp_2.iloc[:, 2:-1].copy()
# Iterate over columns, excluding the first two rows
for col in funding_partners_temp_2_slice.columns:
    for i in range(2, len(funding_partners_temp_2_slice)):
        try:
            # Convert column to numeric type using .at
            funding_partners_temp_2_slice[col] = pd.to_numeric(funding_partners_temp_2_slice[col])
            funding_partners_temp_2_slice.at[i, col] = (funding_partners_temp_2_slice.at[i, col] * funding_partners_temp_2_slice.at[0, col]) / (1000 * funding_partners_temp_2_slice.at[1, col])
        except (ZeroDivisionError, ValueError):
            funding_partners_temp_2_slice.at[i, col] = np.nan
# Round the values to zero decimal points
funding_partners_temp_2_slice = funding_partners_temp_2_slice.round(0)

# Replace NaN values with 0
funding_partners_temp_2_slice = funding_partners_temp_2_slice.fillna(0)
funding_partners_temp_2_slice = pd.concat([funding_partners_temp_2_slice.drop(funding_partners_temp_2_slice.index[2]), funding_partners_temp_2_slice.iloc[[2]]]).reset_index(drop=True)
funding_temp_3 = pd.concat([funding_temp_1, funding_partners_temp_2_slice[2:].reset_index(drop=True)], axis=1)
# Assigning the cost assoicated to not allocated to any project to fuding for the same actor
funding_temp_3.iloc[-1, 2:]=NFR_cost.iloc[-4, 5:-1].values 


NFR_cost_slice = NFR_cost.iloc[-3:-1,]
NFR_cost_slice = NFR_cost_slice.drop(columns=NFR_cost_slice.columns[-1])
NFR_cost_slice = NFR_cost_slice.drop(columns=NFR_cost_slice.columns[2:5])
NFR_cost_slice= NFR_cost_slice.rename(columns={'Total Cost for the Host Institute': 'Host Institute'}).reset_index(drop=True)

funding_temp_4 = pd.concat([funding_temp_3, NFR_cost_slice], axis=0)
funding_temp_4['Total inkind'] =funding_temp_4.iloc[:, 3:].sum(axis=1)
funding_temp_4.reset_index(drop=True,inplace=True)


NFR_cost_slice_2 = NFR_cost.iloc[:-1, -1:]# taking only the cost column (the last column)
funding_temp_5 = pd.concat([funding_temp_4, NFR_cost_slice_2], axis=1)

## adding other funding soruces
funding_temp_6 = funding_temp_5.join(projects_ext_fn, how='left').fillna(0)

## adding other fAid Intensity Limit
projects_assigned_aid=projects_assigned.copy()
projects_assigned_aid['Aid Intensity Limit*****'] = projects_assigned_aid['Type of Research**'].apply(
    lambda x: '65%' if x == 'I' else ('100%' if x == 'F' else None))
funding_temp_7 = funding_temp_6.join(projects_assigned_aid.iloc[:,-1], how='left').fillna('None')

funding_temp_7['RCN Grant']=funding_temp_7['Total Project Cost']-funding_temp_7['Host Institute']-funding_temp_7['Total inkind']-funding_temp_7['Other Public Funding']-funding_temp_7['Other Private Funding']
funding_temp_7['Aid Intensity'] = (funding_temp_7['RCN Grant'] * 100 / (funding_temp_7['Host Institute'] + funding_temp_7['Total inkind'] + funding_temp_7['RCN Grant'])).fillna(0).round().astype(int).astype(str) + '%'

#Reraging the columns
cols = funding_temp_7.columns.tolist()# Get the column names as a list
cols = cols[:2] + [cols[-1], cols[-3]] + cols[2:-3] + cols[-2:-1]# Reorder the columns. Here -1 represents the last column, -3 represents the third last.
funding_temp_7 = funding_temp_7[cols]# Reindex the DataFrame
funding_temp_7.iloc[:, 4:] = funding_temp_7.iloc[:, 4:].round(0).astype(int)

#Droping unnessary columns
funding_temp_7 = funding_temp_7.drop(columns=[funding_temp_7.columns[-5], funding_temp_7.columns[-4]])

funding_temp_7['Indirect state aid ******'] = funding_temp_7['Other Public Funding'].apply(lambda x: 'b' if x == 0 else '')

# Merging Type of Partner information
partner_type_t=partner_type.T
partner_type_t.columns = partner_type_t.iloc[0] 
partner_type_t = partner_type_t.iloc[1:]
partner_type_t = partner_type_t.reset_index(drop=True)

target_df = funding_temp_7 #target dataframe (funding_temp_7)
other_df = partner_type_t# other dataframe (partner_type_t)

missing_columns = [col for col in target_df.columns if col not in other_df.columns]# Get column names in target dataframe not present in other dataframe

# Create a new row using values from the first row of other_df
new_row = pd.DataFrame([other_df.iloc[0][col] if col in other_df.columns else None for col in target_df.columns]).T
new_row.columns = target_df.columns# Rename the columns to match the target dataframe
target_df = pd.concat([target_df, new_row], ignore_index=True)# Append the new row to the target dataframe

# Move the last row to position 0
last_row = target_df.iloc[[-1]]
remaining_rows = target_df.iloc[:-1]
target_df = pd.concat([last_row, remaining_rows], ignore_index=True)
target_df.at[0, 'Project_Name'] = 'Type of partner****'
target_df.at[0, 'Project_code'] = ''


target_df['Project_code'] = target_df['Project_code'].astype(str)# Convert numeric column1 to string

# Merge column1 and column2
target_df['Item'] = target_df['Project_code'] + '-' + target_df['Project_Name']# Convert numeric column1 to string
target_df = target_df.drop(['Project_code', 'Project_Name'], axis=1)# Remove the original columns

# Move the last column to position 0
last_column = target_df.pop(target_df.columns[-1])
target_df.insert(0, last_column.name, last_column)

# Interchange position of column 1 and column 2
target_df = target_df.iloc[:, [0, 2, 1] + list(range(3, len(target_df.columns)))]

# Adding sum of Financing row
df_sub = target_df.iloc[1:, 3:-1]# Exclude columns 0, 1, 2 and last column, and row 0
row_sum = df_sub.sum()# Sum the rows
row_sum_df = pd.DataFrame(row_sum).T# Make the row_sum a DataFrame and transpose it (swap rows and columns) 
target_df = pd.concat([target_df, row_sum_df], ignore_index=True)# Concatenate the original DataFrame with the new DataFrame (which contains the sum)
target_df = target_df.fillna("")# Replace NaN values, if necessary
target_df.iat[-1, 0] = 'Total Financing'

# Adding total fiancing column (for proejcts)
target_df.loc[1:, 'Total Financing'] = target_df.iloc[1:, 3:-1].sum(axis=1)# Sum across the rows excluding the first row and the first three and the last columns
cols = target_df.columns.tolist()# Create a list with the column order
cols.insert(-1, cols.pop(cols.index('Total Financing')))# Move 'Sum' column to the second last position
target_df = target_df[cols]# Reindex DataFrame with the new column order
target_df.iloc[1:, 3:] = target_df.iloc[1:, 3:].applymap(lambda x: '{:.0f}'.format(x) if isinstance(x, float) else x)

SFI_financing= target_df
SFI_financing

/var/folders/35/d6sbh1wd7xsblxcgstl29wc80000gr/T/ipykernel_771/2380318287.py:128: RuntimeWarning: invalid value encountered in scalar divide
  funding_partners_temp_2_slice.at[i, col] = (funding_partners_temp_2_slice.at[i, col] * funding_partners_temp_2_slice.at[0, col]) / (1000 * funding_partners_temp_2_slice.at[1, col])


,Item,Aid Intensity Limit*****,Aid Intensity,Host Institute,Partner 1,Partner 2,Partner 3,Partner 4,Partner 5,Partner 6,Other Public Funding,Other Private Funding,RCN Grant,Total Financing,Indirect state aid ******
0,-Type of partner****,,,,R,P,L,SME,SME,L,,,,NaN,
1,1-Centre Administration,None,0%,101,252,0,0,0,0,0,0,0,0,353,b
2,"2-Equipment, Infrastructure and Laboratory",None,100%,0,0,0,0,0,0,0,0,0,5,5,b
3,1002-Geological Assistant Project Development,None,3%,127,252,60,0,33,110,0,0,0,16,598,b
4,1101-Semantic Plugin for OSDU,65%,62%,89,501,0,0,0,0,36,0,0,1041,1667,b
5,1102-OBDA- SIRIUS Optique Pilot,65%,57%,53,3,0,0,0,0,36,0,0,121,213,b
6,1103-SIRIUS Subsurface Data Laboratory,100%,32%,76,126,0,0,0,110,0,0,0,145,457,b
7,5001-Stakeholder engagement,100%,30%,51,126,0,0,33,0,0,0,0,88,298,b
8,5004-Centre Cultivation,None,36%,38,50,0,0,0,55,0,0,0,78,221,b
9,0-Not Allocated to any project,None,0%,2,0,0,0,0,0,0,0,0,0,2,b


### STEP 10: Export SFI Annual Accounting Report 

In [323]:
# Data Export
SFI_financing.to_excel('SFI_financing.xlsx', index=False)
SFI_cost.to_excel('SFI_cost.xlsx', index=False)
SFI_Running_Cost_Cat.to_excel('SFI_Running_Cost_Cat.xlsx', index=False)